<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/topoflinecondensation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top of line condensation in gas pipelines
In the follwing section we will demonstrate calculation related to top of line condensation in a pipeline where MEG is used as hydrate or corrosion inhibitor.

Litterature:

https://iopscience.iop.org/article/10.1088/1757-899X/1201/1/012082/pdf 

https://www.sciencedirect.com/science/article/pii/S1878535221001313

In [ ]:
%%capture
#We start by installing the neqsim Python package
!pip install neqsim
import neqsim
from neqsim.thermo import fluid, TPflash, printFrame

#Saturate the gas with a MEG/water solution
Select the gas component and set wt% MEG in the bulk glycol solution. For low pressures (<10 bara) the type of gas will have little effect on results.

In [ ]:
wtPercentageMEG = 60.0 #@param {type:"number"}
gascomponent = 'nitrogen' #@param ['methane', 'nitrogen', 'CO2']

In [ ]:
fluid1 = fluid('cpa')
fluid1.addComponent(gascomponent, 5.0, 'kg/hr')
fluid1.addComponent("MEG", wtPercentageMEG, 'kg/hr')
fluid1.addComponent("water", 100.0- wtPercentageMEG, 'kg/hr')
fluid1.setMixingRule(10)

We set the temperature and pressure and run a TPflash to calculate the composition of the gas and liquid. We then display the resulting fluid composition.

In [ ]:
temperature_C = 50.0 #@param {type:"number"}
pressure_bara = 1.01325 #@param {type:"number"}

fluid1.setTemperature(temperature_C, 'C')
fluid1.setPressure(pressure_bara, 'bara')

TPflash(fluid1)

printFrame(fluid1)

| 0                  | 1          | 2                    | 3                    | 4   | 5   | 6               |
|:-------------------|:-----------|:---------------------|:---------------------|:----|:----|:----------------|
|                    | total      | gas                  | aqueous              |     |     |                 |
| nitrogen           | 5.30335E-2 | 9.18891E-1           | 2.15455E-5           |     |     | [mole fraction] |
| MEG                | 2.87226E-1 | 2.84392E-4           | 3.04794E-1           |     |     | [mole fraction] |
| water              | 6.5974E-1  | 8.08245E-2           | 6.95184E-1           |     |     | [mole fraction] |
|                    |            |                      |                      |     |     |                 |
| Density            |            | 1.02687E0            | 1.04048E3            |     |     | [kg/m^3]        |
| PhaseFraction      |            | 5.76925E-2           | 9.42307E-1           |     |     | [mole frac

# Calculation of activity coefficients and water and MEG in gas
The equilibrium can be represented by the equation using vapour pressure of pure components and activity coefficients. The mole fraction of water and MEG in the gas phase can be calculated using the equal fugacity criteria for gas and liquid.

$y_{water}=\frac{x_{water}\times\gamma_{water}*P_{water}}{\varphi_{water}\times P_{total}}$

and

$y_{MEG}=\frac{x_{water}\times\gamma_{MEG}*P_{MEG}}{\varphi_{MEG}\times P_{total}}$

Vapour pressure of MEG can be calculated from the Antonie equation:

https://webbook.nist.gov/cgi/cbook.cgi?ID=C107211&Mask=4

and water:

https://webbook.nist.gov/cgi/cbook.cgi?ID=C7732185&Units=SI&Mask=4#Thermo-Phase


At low pressures fugacity coefficients in will be close to one ($\varphi =1) $. In the following calculations the CPA-EoS in NeqSim are used for the calculations of the fluid phase equilibrium.



In [ ]:
waterInGas = fluid1.getPhase('gas').getComponent('water').getx()*1e2
MEGinGas = fluid1.getPhase('gas').getComponent('MEG').getx()*1e2

print('saturated gas compostion')
print('water in gas ', waterInGas, ' MEG in gas', MEGinGas,' [mole%] ')

water_activity_coefficient = fluid1.getPhase('aqueous').getActivityCoefficient(2)
MEG_activity_coefficient = fluid1.getPhase('aqueous').getActivityCoefficient(1)
print('activity coefficients')
print('water ', water_activity_coefficient, ' MEG ', MEG_activity_coefficient)

saturated gas compostion
water in gas  8.082451279191206  MEG in gas 0.02843921721572875  [mole%] 
activity coefficients
water  0.9566203004345136  MEG  1.0183644488228412


#Calculation of top of line (TOL) fluid composition
The composition of the liquid after condensing the gas on a cold surface can be calulated by running a TPflash calculation on the saturated gas phase. A liquid phase will typically form if the temperature is reduced.

In [ ]:
coldSurfaceTemperature = 20.0 #@param {type:"number"}
coldSurfacePressure = 1.01325 #@param {type:"number"}

gasFluid = fluid1.phaseToSystem('gas')
gasFluid.setTemperature(coldSurfaceTemperature, 'C')
gasFluid.setPressure(coldSurfacePressure, 'bara')
TPflash(gasFluid)
printFrame(gasFluid)

| 0                  | 1           | 2                    | 3                    | 4   | 5   | 6               |
|:-------------------|:------------|:---------------------|:---------------------|:----|:----|:----------------|
|                    | total       | gas                  | aqueous              |     |     |                 |
| nitrogen           | 9.18891E-1  | 9.76669E-1           | 1.18159E-5           |     |     | [mole fraction] |
| MEG                | 2.84392E-4  | 1.9604E-7            | 4.80411E-3           |     |     | [mole fraction] |
| water              | 8.08245E-2  | 2.33304E-2           | 9.95184E-1           |     |     | [mole fraction] |
|                    |             |                      |                      |     |     |                 |
| Density            |             | 1.15514E0            | 9.97748E2            |     |     | [kg/m^3]        |
| PhaseFraction      |             | 9.40841E-1           | 5.91592E-2           |     |     | [

The composition of the condensed phase is

In [ ]:
wtPrecentMEG = gasFluid.getPhase('aqueous').getWtFrac('MEG')*100
wtPrecentwater = gasFluid.getPhase('aqueous').getWtFrac('water')*100
print('composition of condensed phase')
print('water ', wtPrecentwater, ' wt%')
print('MEG ', wtPrecentMEG, ' wt%')

composition of condensed phase
water  98.36220162724736  wt%
MEG  1.6359823420484667  wt%
